## Let's get started using sockets!

Read the Python docs on sockets: https://docs.python.org/3/library/socket.html


Import the socket library

In [1]:
import socket

The Python sockets library allows us to find out what port a service uses.

In [2]:
socket.getservbyname('ssh')

22

You can also do a reverse lookup, finding what service uses a given port.

In [3]:
socket.getservbyport(80)

'http'

The sockets library also provides tools for finding out information about hosts. For example, you can find out about the hostname and IP address of the machine you are currently using.

In [4]:
socket.gethostname()

'LAPTOP-BQP6HJU5'

In [5]:
socket.gethostbyname(socket.gethostname())

'192.168.56.1'

You can also find out about machines that are located elsewhere, assuming you know their hostname. For example:

In [6]:
socket.gethostbyname('google.com')

'216.58.216.174'

In [7]:
socket.gethostbyname('uw.edu')

'128.95.155.198'

In [8]:
socket.gethostbyname('cutecatvideos.net')

'50.97.112.130'

The gethostbyname_ex method of the socket library provides more information about the machines we are exploring:

In [9]:
socket.gethostbyname_ex('cutecatvideos.net')

('cutecatvideos.net', [], ['50.97.112.130'])

In [10]:
socket.gethostbyname_ex('google.com')

('google.com', [], ['216.58.216.174'])

To create a socket, you use the socket method of the socket library. It takes up to three optional positional arguments: family, type, proto, (and fileno)

socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)

The address family should be AF_INET (the default), AF_INET6, AF_UNIX, AF_CAN or AF_RDS. 

The socket type should be SOCK_STREAM (the default), SOCK_DGRAM, SOCK_RAW or perhaps one of the other SOCK_ constants. 

The protocol number is usually zero and may be omitted.

Here we use none to get the default behavior:

In [14]:
my_socket = socket.socket()

In [15]:
my_socket

<socket.socket fd=1236, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>

A socket has some properties that are immediately important to us. These include the family, type and protocol of the socket:

In [16]:
my_socket.family

<AddressFamily.AF_INET: 2>

In [17]:
my_socket.type

<SocketKind.SOCK_STREAM: 1>

In [18]:
my_socket.proto

0

You might notice that the values for these properties are integers. In fact, these integers are constants defined in the socket library.

https://docs.python.org/3/library/socket.html

https://en.wikipedia.org/wiki/Network_socket


Two different types of IP Address

TCP and UDP

## Customizing Sockets

Family, Type, and Protocol are the properties of a socket correspond to the three positional arguments you may pass to the socket constructor.

Using them allows you to create sockets with specific communications profiles:

In [27]:
socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

<socket.socket fd=1228, family=AddressFamily.AF_INET, type=SocketKind.SOCK_DGRAM, proto=17>

## Client Side

We’ve already made a socket foo using the generic constructor without any arguments. We can make a better one now by using real address information from a real server online.

In [22]:
client_socket = socket.socket()

## Connecting a Socket

In [24]:
client_socket.connect(('www.cutecatvideos.com', 80))

* a successful connection returns None
* a failed connection raises an error
* you can use the type of error returned to tell why the connection failed.

## Sending a Message

Send a message to the server on the other end of our connection (we’ll learn in session 2 about the message we are sending):

In [25]:
msg = "GET / HTTP/1.1\r\n"

In [26]:
msg += "Host: 'cutecatvideos.net'\r\n\r\n"

In [27]:
msg = msg.encode('utf8')

In [28]:
msg

b"GET / HTTP/1.1\r\nHost: 'cutecatvideos.net'\r\n\r\n"

In [29]:
client_socket.sendall(msg)

* the transmission continues until all data is sent or an error occurs
* success returns None
* failure to send raises an error
* the type of error can tell you why the transmission failed
* but you cannot know how much, if any, of your data was sent

## Messages are Bytes

One detail from the previous code should stand out:

In [46]:
#msg = msg.encode('utf8')

In [30]:
msg

b"GET / HTTP/1.1\r\nHost: 'cutecatvideos.net'\r\n\r\n"

You can only send bytes through a socket, never unicode

In [31]:
client_socket.sendall(msg.decode('utf8'))

TypeError: a bytes-like object is required, not 'str'

## Receiving a Reply

Whatever reply we get is received by the socket we created. We can read it back out (again, do not type this yet):

In [32]:
response = client_socket.recv(4096)

In [33]:
response[:60]

b'HTTP/1.1 403 Forbidden\r\nCache-Control: private\r\nContent-Type'

* The sole required argument is buffer_size (an integer). It should be a power of 2 and smallish (~4096)
* It returns a byte string of buffer_size (or smaller if less data was received)
* If the response is longer than buffer size, you can call the method repeatedly. The last bunch will be less than buffer size.

## Cleaning Up

In [34]:
client_socket.close()

## Pulling it all together

First, connect and send a message:

In [35]:
info = socket.getaddrinfo('cutecatvideos.net', 'http')

In [36]:
streams = [i for i in info if i[1] == socket.SOCK_STREAM]

In [37]:
sock_info = streams[0]

In [38]:
msg = "GET / HTTP/1.1\r\n"

In [39]:
msg += "Host: cutecatvideos.net\r\n\r\n"

In [40]:
msg = msg.encode('utf8')

In [41]:
client_socket = socket.socket(*sock_info[:3])

In [42]:
client_socket.connect(sock_info[-1])

In [43]:
client_socket.sendall(msg)

Then, receive a reply, iterating until it is complete:

In [44]:
buffsize = 4096
response = b''
done = False
while not done:
    msg_part = client_socket.recv(buffsize)
    if len(msg_part) < buffsize:
        done = True
        client_socket.close()
        response += msg_part

In [45]:
len(response)

509

## Server Side

What about the other half of the equation?

Let’s build a server and see how that part works.

#### Construct a Socket

Again, we begin by constructing a socket. Since we are actually the server this time, we get to choose family, type and protocol:

In [52]:
server_socket = socket.socket(
   ....:     socket.AF_INET,
   ....:     socket.SOCK_STREAM,
   ....:     socket.IPPROTO_TCP)

In [53]:
server_socket

<socket.socket fd=968, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>

#### Bind the Socket

In [57]:
address = ('127.0.0.1', 50000)
server_socket.bind(address)

Terminology Note: In a server/client relationship, the server binds to an address and port. The client connects

#### Listen for Connections

Once our socket is bound to an address, we can listen for attempted connections: